In [ ]:
%pip install requests folium tqdm ratelimit loguru matplotlib basemap numpy

In [ ]:
import json

import requests
from loguru import logger
from ratelimit import limits, sleep_and_retry
from tqdm import tqdm

# 禁止向控制台输出日志
logger.remove()
logger.add('run.log')


# 读取 txt 文件并将每一行存储为一个列表
def read_ips(file_path):
    with open(file_path, 'r') as file:
        ips = [line.strip() for line in file]
    return ips


def process_ips(ips, batch_size=90):
    """
    将 ips 列表切成一批(不超过100个), 并使用 batch_ip_query 函数进行查询
    """
    successful_all, failed_all = [], []

    for i in tqdm(range(0, len(ips), batch_size)):
        ip_list = ips[i:i + batch_size]
        successful, failed = batch_ip_query(ip_list)

        successful_all.extend(successful)
        failed_all.extend(failed)

        # 每一批处理完后，保存到 JSON 文件
        save_list_to_json(successful_all, "successful_ips.json")
        save_list_to_json(failed_all, "failed_ips.json")

    return successful_all, failed_all


@sleep_and_retry
@limits(calls=40, period=60)  # 每分钟限制 45 个
def batch_ip_query(ip_list):
    """
    批量查询 IP 地址信息。

    Args:
        ip_list: 要查询的 IP 地址列表。(最多一百个)

    Returns:
        tuple: 包含两个列表的元组，第一个列表包含成功查询的 IP 地址和其经纬度，格式为 (query, lat, lon)，第二个列表包含失败查询的 IP 地址。
    """
    url = 'http://ip-api.com/batch?fields=57536'

    response = requests.post(url, json=ip_list, timeout=10)
    datas = response.json()

    successful = []
    failed = []

    for data in datas:
        if data['status'] == 'success':
            successful.append((data['query'], data['lat'], data['lon']))
            logger.info(f"IP {data['query'], data['lat'], data['lon']}")
        else:
            failed.append((data['query']))
            logger.warning(f"Query IP {data['query']} failed")

    return successful, failed


def save_list_to_json(data_list, file_path, mode="w"):
    """
    将列表数据保存到 JSON 文件中

    Args:
        data_list: 要保存的列表数据
        file_path: JSON 文件路径
        mode: 文件打开模式，默认为 "w" (覆盖写入)，也可以使用 "a" (追加写入)
    """
    with open(file_path, mode) as f:
        json.dump(data_list, f, indent=4)


In [ ]:
ips = read_ips('malicious_ips.txt')
ips[10:20]

In [ ]:
successful_all, failed_all = process_ips(ips)

In [ ]:
len(successful_all)

In [ ]:
successful_all[:10]

In [ ]:
def load_list_from_json(file_path):
    """
    从 JSON 文件中读取列表数据
    
    Args:
        file_path: JSON 文件路径
    
    Returns:
        读取到的列表数据
    """
    with open(file_path, "r") as f:
        return json.load(f)

In [ ]:
successful_all = load_list_from_json('successful_ips.json')
successful_all[:10]

In [ ]:
coordinates = [(lat, lon) for _, lat, lon in successful_all]
coordinates[:10]

In [ ]:
import folium
from folium.plugins import HeatMap

m = folium.Map()
HeatMap(coordinates).add_to(m)
m

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap

# 创建地图
plt.figure(figsize=(16, 12))
m = Basemap(projection='mill', llcrnrlat=-60, urcrnrlat=90, llcrnrlon=-180, urcrnrlon=180)

# 绘制海岸线和国家边界
m.drawcoastlines()
m.drawcountries()

# 绘制位置点
for lat, lon in coordinates:
    x, y = m(lon, lat)
    m.plot(x, y, 'bo', markersize=4)

# 显示地图
plt.title('Malicious IP Geo Locations on World Map')
plt.show()
plt.savefig('Malicious IP Geo Locations on World Map')

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap

# 创建地图
plt.figure(figsize=(12, 8))
m = Basemap(projection='merc', llcrnrlat=16, urcrnrlat=54, llcrnrlon=72, urcrnrlon=136, resolution='l')

# 绘制海岸线和国家边界
m.drawcoastlines()
m.drawcountries()

# 绘制位置点
for lat, lon in coordinates:
    x, y = m(lon, lat)
    m.plot(x, y, 'bo', markersize=5)

# 显示地图
plt.title('Malicious IP Geo Locations on China Map')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
import numpy as np

# 创建地图
plt.figure(figsize=(12, 8))
m = Basemap(projection='merc', llcrnrlat=16, urcrnrlat=54, llcrnrlon=72, urcrnrlon=136, resolution='l')

# 加载中国省份边界数据
# https://gadm.org/download_country.html
# 下载链接: https://geodata.ucdavis.edu/gadm/gadm4.1/shp/gadm41_CHN_shp.zip
shp_info = m.readshapefile('gadm41_CHN_shp/gadm41_CHN_1', 'states', drawbounds=True)

# 绘制省份界线
for shapedict, seg in zip(m.states_info, m.states):
    # 获取省份名称
    # province = shapedict['NAME_1']
    # 将 seg 转换为 NumPy 数组
    seg = np.array(seg)
    # 绘制省份边界
    m.plot(seg[:, 0], seg[:, 1], 'k-', linewidth=0.5)
    # 在省份中心添加文字
    # 省份中心点坐标计算
    x, y = seg.mean(axis=0)
    # 使用 plt.text 添加文字
    # plt.text(x, y, province, ha="center", fontsize=8)

# 绘制海岸线和国家边界
m.drawcoastlines()
m.drawcountries()

# 绘制位置点
for lat, lon in coordinates:
    x, y = m(lon, lat)
    m.plot(x, y, 'bo', markersize=5)

# 显示地图
plt.title('Malicious IP Geo Locations on China Map')
plt.show()

In [ ]:
import folium

# 创建一个Folium地图对象
m = folium.Map(location=[20, 0], zoom_start=2)

# 添加点到地图上
for lat, lon in coordinates:
    folium.Marker([lat, lon]).add_to(m)

# 保存地图到HTML文件
# m.save("map.html")

# 如果你在Jupyter Notebook中，可以直接显示
m
